# ASP Stereo Output Plotting
## Examples for BlackSky Easton Glacier test case (n=20)

David Shean  
1/2/23

In [ ]:
import os
import glob

import numpy as np
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
from mpl_toolkits.axes_grid1 import make_axes_locatable
import rasterio
from rasterio.windows import Window
from osgeo import gdal

In [ ]:
outdir = '/Users/dshean/scr/BlackSky/EastonGlacier_20220918-20221012/non-ortho/stereo_ba_all_tri_weight/BSG-112-20220918-173557-38516577__BSG-113-20220918-175217-38567076'

In [ ]:
outdir = '/Users/dshean/scr/BlackSky/EastonGlacier_20220918-20221012/non-ortho/stereo_ba_all_tri_weight/BSG-108-20221005-191015-40132991__BSG-118-20221012-202648-41004143'

In [ ]:
cd $outdir

In [ ]:
def symm_clim(clim):
    abs_max = np.max(np.abs(clim))
    return (-abs_max, abs_max)

def get_clim(ar, perc=(2,98), symm=False):
    try:
        clim = np.percentile(ar.compressed(), perc)
    except:
        clim = np.percentile(ar, perc)
    if symm:
        clim = symm_clim(clim)
    return clim

#Generalize for input list, not just two images
def find_common_clim(im1, im2, symm=False):
    clim1 = get_clim(im1)
    clim2 = get_clim(im2)
    clim = (np.min([clim1[0],clim2[0]]), np.max([clim1[1],clim2[1]]))
    if symm:
        clim = symm_clim(clim)
    return clim

def read_array(fn, b=1):
    ds = rasterio.open(fn)
    a = ds.read(b, masked=True)
    ndv = get_ndv(ds)
    #The stddev grids have nan values, separate from nodata
    ma = np.ma.fix_invalid(np.ma.masked_equal(a, ndv))
    return ma

def get_ndv(ds):
    ndv = ds.nodatavals[0]
    if ndv == None:
        ndv = ds.read(1, window=Window(0, 0, 1, 1)).squeeze()
    return ndv

In [ ]:
def get_cbar_extend(a, clim=None):
    """
    Determine whether we need to add triangles to ends of colorbar
    """
    if clim is None:
        clim = get_clim(a)
    extend = 'both'
    if a.min() >= clim[0] and a.max() <= clim[1]:
        extend = 'neither'
    elif a.min() >= clim[0] and a.max() > clim[1]:
        extend = 'max'
    elif a.min() < clim[0] and a.max() <= clim[1]:
        extend = 'min'
    return extend

In [ ]:
def plot_ar(im, ax, cmap='inferno', clim=None, clim_perc=(2,98), label=None, cbar=True, alpha=1):
    if clim is None:
        get_clim(im, clim_perc)
    m = ax.imshow(im, cmap=cmap, clim=clim, alpha=alpha, interpolation='none')
    if cbar:
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="4%", pad="2%")
        cb = plt.colorbar(m, cax=cax, ax=ax, extend=get_cbar_extend(im, clim))
        cax.set_ylabel(label)
    ax.set_facecolor("0.5")
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
def plot_corr_results(outdir):
    f, axa = plt.subplots(3, 2, figsize=(7.5,10), dpi=220)
    axa = axa.ravel()
      
    l_img_fn = glob.glob('*-L.tif')[0]
    l_img_ma = read_array(l_img_fn)
    
    r_img_fn = glob.glob('*-R.tif')[0]
    r_img_ma = read_array(r_img_fn)
    
    disp_fn = glob.glob('*-F.tif')[0]
    dx_ma = read_array(disp_fn, 1)
    dy_ma = read_array(disp_fn, 2)
    disp_clim = find_common_clim(dx_ma, dy_ma, symm=True)
    
    l_stddev_fn = glob.glob('*-L_stddev_filter_output.tif')
    if l_stddev_fn:
        l_stddev_fn = l_stddev_fn[0]
        l_stddev_ma = read_array(l_stddev_fn)
        r_stddev_fn = glob.glob('*-R_stddev_filter_output.tif')[0]
        r_stddev_ma = read_array(r_stddev_fn)
        stddev_clim = list(find_common_clim(l_stddev_ma, r_stddev_ma))
        stddev_clim[0] = 0
        plot_ar(l_stddev_ma, ax=axa[2], clim=stddev_clim, cmap='inferno', label='std filter (px)')
        plot_ar(r_stddev_ma, ax=axa[3], clim=stddev_clim, cmap='inferno', label='std filter (px)')
    
    plot_ar(l_img_ma, ax=axa[0], cmap='gray', clim_perc=(0,100))
    plot_ar(r_img_ma, ax=axa[1], cmap='gray', clim_perc=(0,100))
    
    plot_ar(dx_ma, ax=axa[4], clim=disp_clim, cmap='RdBu', label='x disparity (px)')
    plot_ar(dy_ma, ax=axa[5], clim=disp_clim, cmap='RdBu', label='y disparity (px)')
    
    plt.suptitle(os.path.split(outdir)[-1])
    plt.tight_layout()
    out_fn = l_img_fn.split('-L.tif')[0]+'-corr_report.png'
    plt.savefig(out_fn)

In [ ]:
plot_corr_results(outdir)

In [ ]:
def plot_dem_results(outdir, hillshade=True):
    f, axa = plt.subplots(1, 5, figsize=(10,7.5), dpi=220)
    axa = axa.ravel()

    ortho_fn = glob.glob('*-L_ortho.tif')
    if ortho_fn:
        ortho_fn = ortho_fn[0]
        ortho_ma = read_array(ortho_fn)
        plot_ar(ortho_ma, ax=axa[0], cmap='gray')
    else:
        axa[0].axis('off')
        
    ortho_fn = glob.glob('*-R_ortho.tif')
    if ortho_fn:
        ortho_fn = ortho_fn[0]
        ortho_ma = read_array(ortho_fn)
        plot_ar(ortho_ma, ax=axa[1], cmap='gray')
    else:
        axa[1].axis('off')
    
    dem_fn = glob.glob('*-DEM.tif')[0]
    dem_ma = read_array(dem_fn)
    with rasterio.open(dem_fn) as ds:
        dem_gsd = ds.res[0]
    
    if hillshade:
        hs_fn = glob.glob('*-DEM_hs.tif')
        if hs_fn:
            hs_fn = hs_fn[0]
            hs_ma = read_array(hs_fn)
        else:
            dem_ds = gdal.Open(dem_fn)
            hs_ds = gdal.DEMProcessing('', dem_ds, 'hillshade', format='MEM', computeEdges=True)
            hs_ma = np.ma.masked_equal(hs_ds.ReadAsArray(), 0)
        plot_ar(hs_ma, ax=axa[2], cmap='gray', clim_perc=(5,95), cbar=False)
    
    #axa[4].set_title(os.path.split(dem_fn)[-1])
    plot_ar(dem_ma, ax=axa[2], cmap='viridis', label='Elevation (m HAE)', alpha=0.5)
    
    scalebar = ScaleBar(dem_gsd)
    axa[2].add_artist(scalebar)
    
    #This is not generated by default (requires point2dem --errorimage)
    error_fn = glob.glob('*-IntersectionErr.tif')
    if error_fn:
        error_fn = error_fn[0]
        error_ma = read_array(error_fn)
        plot_ar(error_ma, ax=axa[3], clim=get_clim(error_ma), cmap='inferno', label='Intersection Error (m)')
    else:
        axa[3].axis('off')
    
    #This is not generated by default (requires point2dem --errorimage)
    diff_fn = glob.glob('*-DEM*diff.tif')
    if diff_fn:
        diff_fn = diff_fn[0]
        diff_ma = read_array(diff_fn)
        plot_ar(diff_ma, ax=axa[4], clim=get_clim(diff_ma, symm=True), cmap='RdBu', label='Refdem diff (m)')
    else:
        axa[4].axis('off')
    
    f.suptitle(os.path.split(outdir)[-1])
    f.tight_layout()
    out_fn = dem_fn.split('-DEM.tif')[0]+'-stereo_report.png'
    plt.savefig(out_fn, bbox_inches='tight')

In [ ]:
plot_dem_results(outdir)